# Fetch and Clean the Scraped Data

- Fetch data using pandas ```read_csv()``` function
- Check the data and perform the necessary cleaning

In [142]:
import pandas as pd

asn_data = pd.read_csv('asn_dataset.csv')
asn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8288 entries, 0 to 8287
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   acc. date   8288 non-null   object 
 1   type        8288 non-null   object 
 2   reg.        8053 non-null   object 
 3   operator    8234 non-null   object 
 4   fat.        8206 non-null   object 
 5   location    8266 non-null   object 
 6   Unnamed: 6  0 non-null      float64
 7   dmg         8284 non-null   object 
 8   Unnamed: 8  0 non-null      float64
 9   Unnamed: 9  0 non-null      float64
dtypes: float64(3), object(7)
memory usage: 647.6+ KB


### From the info above

- We have 10 columns, but column 6, 8, and 9 are unnamed and has 0 non-null values.
    - Remove columns 6, 8, 9
- We will then take a look at the top 10 rows to clean the data further

In [143]:
# Remove columns 6, 8, 9
asn_data = asn_data.drop(['Unnamed: 6','Unnamed: 8','Unnamed: 9'],axis=1)

In [144]:
# See top 10 rows
asn_data.head(10)


,acc. date,type,reg.,operator,fat.,location,dmg
0,2 Jan 1995,Boeing 737-298C,9Q-CNI,Air Zaire,0,Kinshasa-N'Djili Airport (FIH),w/o
1,2 Jan 1995,Cessna 208 Caravan I,N242SS,Taquan Air Service,0,"Craig, AK",sub
2,3 Jan 1995,de Havilland Canada DHC-6 Twin Otter 310,P2-IAA,Islands Nationair,0,Bili,w/o
3,4 Jan 1995,Fokker 50?,NaN,Sudan Airways,0,Port Sudan Airport (PZU),non
4,5 Jan 1995,Fokker 50,LN-BBA,"Braathens SAFE, lsf Norwegian Air Shuttle",0,Ã lesund-Vigra Airport (AES),sub
5,5 Jan 1995,Lockheed L-1329-25 JetStar II,1003,Imperial Iranian Air Force - IIAF,12,near Isfahan-Shahid Beheshti Airport (IFN),w/o
6,6 Jan 1995,McDonnell Douglas MD-88,N981DL,Delta Air Lines,0,"Monroe, LA",non
7,8 Jan 1995,Cessna 208 Caravan I,VH-MMV,A. McVinish,0,"Toogoolawah Airport, QLD",sub
8,10 Jan 1995,Rockwell Sabreliner 60,N771WW,Wallace's Bookstore Inc,0,"Lexington-Blue Grass Airport, KY (LEX)",w/o
9,10 Jan 1995,Douglas DC-9-14,HK-3839X,Intercontinental de AviaciÃ³n,51,near Maria La Baja,w/o


### Modifications required:
- Remove space and periods(.) from the columns names
- Change the data type of dates to datetime
- Change damage_type to category
- Expand the damage type to more understandable format, i.e. Replace w/o - Write Off, sub - Substantial, non - None, min - Minor, etc

In [145]:
# Clean column names

asn_data.rename(columns={'acc. date':'accident_date','type':'aircraft_type','reg.':'registration','fat.':'fatalities','dmg':'damage_type'}, inplace=True)

asn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8288 entries, 0 to 8287
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   accident_date  8288 non-null   object
 1   aircraft_type  8288 non-null   object
 2   registration   8053 non-null   object
 3   operator       8234 non-null   object
 4   fatalities     8206 non-null   object
 5   location       8266 non-null   object
 6   damage_type    8284 non-null   object
dtypes: object(7)
memory usage: 453.4+ KB


In [146]:
# Change the accident_date data type to date time
asn_data['accident_date'] = pd.to_datetime(asn_data['accident_date'],errors='coerce')
# Change the damage_type to category
asn_data['damage_type'] = asn_data['damage_type'].astype('category')


In [147]:
asn_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8288 entries, 0 to 8287
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   accident_date  8195 non-null   datetime64[ns]
 1   aircraft_type  8288 non-null   object        
 2   registration   8053 non-null   object        
 3   operator       8234 non-null   object        
 4   fatalities     8206 non-null   object        
 5   location       8266 non-null   object        
 6   damage_type    8284 non-null   category      
dtypes: category(1), datetime64[ns](1), object(5)
memory usage: 396.9+ KB


In [148]:
# get all the unique damage type
print(asn_data['damage_type'].unique())

['w/o', 'sub', 'non', 'mis', 'unk', 'min', NaN]
Categories (6, object): ['min', 'mis', 'non', 'sub', 'unk', 'w/o']


### Replace the damage types as follows

- w/o --> Write Off
- sub --> Substantial
- non --> None
- mis --> Aircraft Missing (written off)
- unk --> Unknown
- min --> Minor Damage

Note: dropped the empty values

In [149]:
asn_data['damage_type'] = asn_data['damage_type'].replace({
    'w/o':'Write-Off',
    'sub':'Substantial',
    'non':'None (No Damage)',
    'mis':'Aircraft Missing',
    'unk':'Unknown',
    'min':'Minor Damage'
})


C:\Users\BHAVYA\AppData\Local\Temp\ipykernel_11332\1527741044.py:1: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  asn_data['damage_type'] = asn_data['damage_type'].replace({


In [150]:
asn_data.describe(include='all')

,accident_date,aircraft_type,registration,operator,fatalities,location,damage_type
count,8195,8288,8053,8234,8206,8266,8284
unique,NaN,2029,7801,4164,268,6040,6
top,NaN,Cessna 208B Grand Caravan,-,private,0,"Chicago-O'Hare International Airport, IL (ORD/...",Substantial
freq,NaN,287,5,151,5610,32,3595
mean,2009-03-20 22:05:25.954850560,NaN,NaN,NaN,NaN,NaN,NaN
min,1995-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
25%,2001-07-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
50%,2008-08-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
75%,2016-09-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
max,2025-07-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
# Output the cleaned data to a new CSV file
asn_data.to_csv('cleaned_asn_dataset.csv', index=False)